In [37]:
from keras.layers import Dense, Convolution2D, Activation, Flatten, Conv2D
from keras.layers import Dropout
from keras.models import Model, Sequential
from keras.datasets import mnist

In [7]:
data = mnist.load_data()

In [8]:
def make_untrainable(model):
    pass

def make_trainable(model):
    pass

In [9]:
from keras.utils import to_categorical

In [16]:
# preprocess
(x_train, y_train), (x_test, y_test) = data
(y_train, y_test) = map(lambda x:to_categorical(x, 10), [y_train, y_test])
x_train = x_train.reshape((-1, 28 * 28))
x_test = x_test.reshape((-1, 28 * 28))

In [17]:
# testing data
x_train.shape

(60000, 784)

In [64]:
# Discrimnator 

discriminator = Sequential()
discriminator.add(Dense(units = 28 * 28, input_dim = 784))
discriminator.add(Activation('tanh'))
discriminator.add(Activation('relu'))
discriminator.add(Dropout((14 * 14)/(28 * 28)))
discriminator.add(Dense(units = 14 * 14, input_dim = 14 * 14))
discriminator.add(Activation('tanh'))
discriminator.add(Activation('relu'))
discriminator.add(Dropout((7 * 7)/(14 * 14)))
discriminator.add(Dense(units = 7 * 7, input_dim = 7 * 7, activation = 'tanh'))
discriminator.add(Dense(units = 7 * 7, input_dim = 7 * 7, activation = 'relu'))
discriminator.add(Dense(units = 10, input_dim = 7 * 7, activation = 'sigmoid'))


discriminator.summary()
discriminator.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 784)               615440    
_________________________________________________________________
activation_77 (Activation)   (None, 784)               0         
_________________________________________________________________
activation_78 (Activation)   (None, 784)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_70 (Dense)             (None, 196)               153860    
_________________________________________________________________
activation_79 (Activation)   (None, 196)               0         
_________________________________________________________________
activation_80 (Activation)   (None, 196)               0         
__________

In [65]:
# training descriminator

discriminator.fit(x_train, y_train, batch_size= 50, verbose=1, epochs = 100, shuffle = True)

Epoch 1/100
60000/60000 [==============================] - 29s - loss: 0.7933 - acc: 0.7333    
Epoch 2/100
60000/60000 [==============================] - 29s - loss: 0.4525 - acc: 0.8563    
Epoch 3/100
60000/60000 [==============================] - 29s - loss: 0.3732 - acc: 0.8840    
Epoch 4/100
60000/60000 [==============================] - 28s - loss: 0.3376 - acc: 0.8953    
Epoch 5/100
60000/60000 [==============================] - 26s - loss: 0.3098 - acc: 0.9032    
Epoch 6/100
60000/60000 [==============================] - 25s - loss: 0.2997 - acc: 0.9070    
Epoch 7/100
60000/60000 [==============================] - 27s - loss: 0.2840 - acc: 0.9119    
Epoch 8/100
60000/60000 [==============================] - 25s - loss: 0.2745 - acc: 0.9143    
Epoch 9/100
60000/60000 [==============================] - 26s - loss: 0.2609 - acc: 0.9186    
Epoch 10/100
60000/60000 [==============================] - 25s - loss: 0.2499 - acc: 0.9225    
Epoch 11/100
60000/60000 [=============

60000/60000 [==============================] - 37s - loss: 0.1118 - acc: 0.9657    
Epoch 85/100
60000/60000 [==============================] - 36s - loss: 0.1105 - acc: 0.9656    
Epoch 86/100
60000/60000 [==============================] - 36s - loss: 0.1100 - acc: 0.9654    
Epoch 87/100
60000/60000 [==============================] - 36s - loss: 0.1093 - acc: 0.9659    
Epoch 88/100
60000/60000 [==============================] - 36s - loss: 0.1105 - acc: 0.9655    
Epoch 89/100
60000/60000 [==============================] - 37s - loss: 0.1096 - acc: 0.9650    
Epoch 90/100
60000/60000 [==============================] - 37s - loss: 0.1088 - acc: 0.9663    
Epoch 91/100
60000/60000 [==============================] - 35s - loss: 0.1086 - acc: 0.9661    
Epoch 92/100
60000/60000 [==============================] - 38s - loss: 0.1090 - acc: 0.9666    
Epoch 93/100
60000/60000 [==============================] - 37s - loss: 0.1073 - acc: 0.9664    
Epoch 94/100
60000/60000 [=================

In [49]:
# testing descriminator
print("\n",descriminator.evaluate(x_test, y_test))
print(" Pretty good :)")


 9696/10000 [============================>.] - ETA: 0s
 [0.072622292394097898, 0.97989999999999999]
 Pretty good :)


In [55]:
descriminator.save_weights("discriminator.hdf")

In [ ]:
# Generator


In [ ]:
# training loop

In [50]:
# tests
